Installing requirements

In [ ]:
!pip install pandas numpy hazm tqdm openpyxl scikit-learn transformers sentencepiece
!pip install scikit-learn
!pip install summa

In [ ]:
!pip install ace_tools

Importing Libraries

In [ ]:
import os, re
import pandas as pd
from tqdm import tqdm
from hazm import Normalizer, sent_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
import numpy as np

# Fine-tuned GLOT500

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd '/content/drive/MyDrive/NLP/digikala_data'

/content/drive/MyDrive/NLP/digikala_data


In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("highest_3000_info_data_QA_patched.csv")

len(df)

3000

In [ ]:
df_test = df.iloc[:20]

len(df_test)

20

In [ ]:
df_test

,product_id,title_fa,brand,category1,category2,price,min_price_last_month,rate_mean,rate_cnt,likes,dislikes,pros_sample,cons_sample,comments_summary,full_summary,qa_pairs,row_index
0,3331738,لیزر موهای زائد مدل A110,متفرقه,اصلاح موی بدن بانوان,NaN,19990000,23100000,2.834783,23,180,7,سادگی کار\nکیفیت ساخت\nعینک مخصوص\nتاثیر خوب\n...,عدم استفاده از منبع لیزر\nعدم کارایی بعد از سه...,نظر 1: خوب\nنظر 2: در زمان دراز جواب میده!\nنظ...,خلاصه نظرات:\nنظر 1: خوب\nنظر 2: در زمان دراز ...,Q1: آیا استفاده از زومیو برای پوست‌های گندمی م...,2
1,6007142,پیانو دیجیتال یاماها مدل DGX-670,یاماها,سازهای صفحه کلیددار,پیانو,501600000,0,3.992593,27,113,8,امکانات عالی\nصدای طبیعی\nظاهری زیبا\nکیفیت سا...,صندلی نداره\nپدال معمولی داره\nکلاویه میتونست ...,نظر 1: من حدود دو هفته هستش که این پیانو را خر...,خلاصه نظرات:\nنظر 1: من حدود دو هفته هستش که ا...,Q1: پیانو چه ویژگی‌هایی دارد که آن را خاص می‌ک...,3
2,1637173,تصفیه کننده هوا آلماپرایم مدل AP-151,آلماپرایم,سلامت محیط,تصفیه کننده هوا,42750000,0,3.475000,16,54,1,عملکرد خوب در کار بیش از ۸ ساعت مداوم\nقابلیت ...,صدای بالا در هنگام کار در اتاق با دور موتور با...,نظر 1: از نظر عملکردش در تصفیه هوا نمیشه نظر د...,خلاصه نظرات:\nنظر 1: از نظر عملکردش در تصفیه ه...,Q1: چه مشکلاتی در طراحی آلماپرایم وجود دارد؟\n...,4
3,1637173,تصفیه کننده هوا آلماپرایم مدل AP-151,آلماپرایم,سلامت محیط,تصفیه کننده هوا,42750000,0,3.475000,16,54,1,عملکرد خوب در کار بیش از ۸ ساعت مداوم\nقابلیت ...,صدای بالا در هنگام کار در اتاق با دور موتور با...,نظر 1: از نظر عملکردش در تصفیه هوا نمیشه نظر د...,خلاصه نظرات:\nنظر 1: از نظر عملکردش در تصفیه ه...,Q1: چه مشکلاتی در طراحی آلماپرایم وجود دارد؟\n...,5
4,8110629,ماسک دور چشم ژنو بایوتیک مدل 01 حجم 30 میلی لیتر,ژنوبایوتیک,مراقبت پوست,ماسک صورت و بدن,2350000,2296000,3.576923,26,111,0,ابرسانی\nخنک میکنه\nزود جذبه\nبسته بندی جالب\n...,عدم تاثیر\nرنگ مصنوعی ابی کم رنگ\nآبرسانی کم\n...,نظر 1: خوب بود راضی بودم\nنظر 2: خوبه. بد نیس....,خلاصه نظرات:\nنظر 1: خوب بود راضی بودم\nنظر 2:...,Q1: ماسک خواب مخصوص دورچشم چه ترکیباتی دارد؟\n...,6
5,8110629,ماسک دور چشم ژنو بایوتیک مدل 01 حجم 30 میلی لیتر,ژنوبایوتیک,مراقبت پوست,ماسک صورت و بدن,2350000,2296000,3.576923,26,111,0,ابرسانی\nخنک میکنه\nزود جذبه\nبسته بندی جالب\n...,عدم تاثیر\nرنگ مصنوعی ابی کم رنگ\nآبرسانی کم\n...,نظر 1: خوب بود راضی بودم\nنظر 2: خوبه. بد نیس....,خلاصه نظرات:\nنظر 1: خوب بود راضی بودم\nنظر 2:...,Q1: ماسک خواب مخصوص دور چشم چه تأثیری بر روی پ...,7
6,73428,میکروفن کندانسور رود مدل NT-USB,رود,لوازم جانبی آلات موسیقی,میکروفون حرفه‌ای,109000000,109220000,4.221667,30,153,16,کیفیت مقبول\nهمراه داشتن پاپ فیلتر\nاتصال راحت...,قیمت بالا\nدریافت بیشتر فرکانس های بالاتر\nمن ...,نظر 1: چند روز پیش به دستم رسید. کیفیت خوب و ق...,خلاصه نظرات:\nنظر 1: چند روز پیش به دستم رسید....,Q1: کیفیت Rode NT USB چگونه است؟\nA1: کیفیت Ro...,8
7,9405862,میکروفون استودیویی جِم آدیو مدل GA-800 به همرا...,متفرقه,تجهیزات استودیویی,میکروفون استودیویی,46780000,46780000,4.125000,8,21,6,بسته بندی شیک\nموارد استفاده فراوان\nمیکروفون ...,همراهش دستگاه فانتوم نبود,نظر 1: با توجه به اینکه با نیومن کار کردم و سو...,خلاصه نظرات:\nنظر 1: با توجه به اینکه با نیومن...,Q1: آیا میکروفون و کارت صدای این محصول کیفیت ص...,9
8,9705686,سیستم عامل ویندوز 10 نسخه پرو لایسنس ریتیل نشر...,متفرقه,نرم افزار,سیستم عامل,5491000,3850000,4.482759,29,139,30,اصل بود\nسریع کد لایسنس ارسال شد\nکد لایسنس اخ...,کاغذ بی ارزش در بسته بندی بد\nاز طریق پیامک که...,نظر 1: به نظرم ادم اینو بگیره تا ویندوز ۱۱ باگ...,خلاصه نظرات:\nنظر 1: به نظرم ادم اینو بگیره تا...,Q1: چه تجربیاتی کاربران از نصب ویندوز ۱۰ پرو ب...,10
9,56792,ادو تویلت مردانه ایسی میاک مدل L’Eau d’Issey P...,ایسی میاک,عطر و ادکلن مردانه,NaN,25114800,24085100,3.010714,28,76,34,ماندگاری بالا\nبوی بهشت میده! (دوست دارم یه رو...,پخش بو یکم کم\nعادی شدن بو برای بینی استفاده ک...,نظر 1: کیفیت خوبی دارد\nنظر 2: باسپاس از شما\n...,خلاصه نظرات:\nنظر 1: کیفیت خوبی دارد\nنظر 2: ب...,Q1: ماندگاری عطر چه طور است؟\nA1: ماندگاری عطر...,11


In [ ]:
data = []

for _, row in df_test.iterrows():
    summary = row["full_summary"]
    qa_block = row["qa_pairs"]

    if not isinstance(qa_block, str):
        continue

    for pair in qa_block.strip().split("\n\n---\n\n"):
        lines = pair.strip().split("\n")
        q_line = next((l for l in lines if l.startswith("Q")), None)
        a_line = next((l for l in lines if l.startswith("A")), None)
        if q_line and a_line:
            question = q_line.split(":", 1)[1].strip()
            data.append({"question": question, "summary": summary})

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

from sentence_transformers import SentenceTransformer, InputExample, losses, models
from torch.utils.data import DataLoader

train_examples = [
    InputExample(texts=[row["question"], row["summary"]])
    for row in data
]

word_embedding_model = models.Transformer("cis-lmu/glot500-base")
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(), pooling_mode="cls")

model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
train_loss = losses.MultipleNegativesRankingLoss(model)

model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=3,
    warmup_steps=100,
    show_progress_bar=True,
    output_path="./glot500_contrastive_model",
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of XLMRobertaModel were not initialized from the model checkpoint at cis-lmu/glot500-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Step,Training Loss


In [ ]:
summaries = df["full_summary"].astype(str).tolist()
summary_embeddings = model.encode(summaries, convert_to_tensor=True)

query = "کدام تصفیه کننده هوا مناسب ماشین است؟"
query_embedding = model.encode(query, convert_to_tensor=True)

import torch
cosine_scores = torch.nn.functional.cosine_similarity(query_embedding, summary_embeddings)

# top-3
top_results = torch.topk(cosine_scores, k=3)

for score, idx in zip(top_results.values, top_results.indices):
    print(f"\n Score: {score.item():.4f}")
    print(f"Summary:\n{summaries[idx]}")

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)



 Score: 0.9835
Summary:
خلاصه نظرات:
نظر 1: به عنوان یک نوازنده و مدرس ساز واقعا متاسف شدم از خرید خودم. سعی کنید از یک برند با کیفیت خرید و حمایت کنید نه این تخته پاره ها. در نگاه اول کیف یاز بسیار بی کیفیت و با عجله ساخته شده. چروک و کج و کوله. با باز کردن کیف متوجه شکستن خرک ساز شدم. وقتی به درب کیف نگاه کردم دیدم که علت شکستن خرک برخورد با درب کیف بوده و صفحه ساز کاملا تحت فشار بودا و احتمالا صفحه ساز هم اسیب دیده باشه. یک خرک جایگزین برای ساز تراش دادم ولی متاسفانه صدای دوتار ترکمنی میده بیشتر تا ویولن. گریف ساز ضخمی و معیوب و سیمگیر بد ریخت و کلفت انگار که کار دستی بچه باشه. رنگ ساز اسیب دیده و ناشیانه ترمیمش کردند انگار ساز دست دوم هست. سه زه ساز انگار نقاشی شده و د اکثر جاها فاصله کم و زیاده.. حلزونز ساز فقط شبیه به حلزونی ویولن هست. قسمتهایی از ساز بدون دلیل رنگ مشکی خورده که ظاهر ساز را افتضاح کرده. زیر چانه پوسته پوسته شده و ازار میده پوست صورت را. دسته ساز هم شبیه التو هست و خیلی بزرگ و بد قواره هست. رنگ داخل کیف مثل خاک جدا میشه و همه جای ساز را کثیف میکنه. ارشه ساز با عج

# Improved Fine-tuned GLOT500 (Senetence-BERT style, Dual encoder)

In [ ]:

!pip install -q sentence-transformers

import pandas as pd
import os
from sentence_transformers import SentenceTransformer, InputExample, losses, models
from torch.utils.data import DataLoader

os.environ['WANDB_DISABLED'] = 'true'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 97.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 64.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 78.2 MB/s eta 0:00:00


In [ ]:
# STEP 2: Load your dataset
CSV_PATH = "highest_3000_info_data_QA_patched.csv"

# STEP 3: Parse question-summary pairs
df = pd.read_csv(CSV_PATH)

In [ ]:
df = df.iloc[:20]

len(df)

20

In [ ]:
data = []

for _, row in df.iterrows():
    summary = row.get("full_summary", None)
    qa_block = row.get("qa_pairs", None)

    if not isinstance(summary, str) or not isinstance(qa_block, str):
        continue

    for pair in qa_block.strip().split("\n\n---\n\n"):
        lines = pair.strip().split("\n")
        q_line = next((l for l in lines if l.startswith("Q")), None)
        a_line = next((l for l in lines if l.startswith("A")), None)
        if q_line and a_line:
            question = q_line.split(":", 1)[1].strip()
            data.append(InputExample(texts=[question, summary]))

print(f"Total usable (question, summary) pairs: {len(data)}")

Total usable (question, summary) pairs: 100


In [ ]:
# STEP 4: Load GLOT500 model as encoder
word_model = models.Transformer("cis-lmu/glot500-base")
pooling_model = models.Pooling(
    word_model.get_word_embedding_dimension(),
    pooling_mode="cls"
)
model = SentenceTransformer(modules=[word_model, pooling_model])

# STEP 5: Prepare dataloader & loss
train_dataloader = DataLoader(data, shuffle=True, batch_size=16)
train_loss = losses.MultipleNegativesRankingLoss(model=model)

# STEP 6: Fine-tune the model
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=3,
    warmup_steps=100,
    output_path="/content/glot500_contrastive_model",
    show_progress_bar=True
)

# STEP 6.5: Sanity check after training
# Select a random question-summary pair and verify similarity has increased
from random import choice
sample = choice(data)
model = SentenceTransformer("/content/glot500_contrastive_model")  # Reload trained model

sample_question = sample.texts[0]
sample_summary = sample.texts[1]

q_emb = model.encode(sample_question, convert_to_tensor=True, normalize_embeddings=True)
s_emb = model.encode(sample_summary, convert_to_tensor=True, normalize_embeddings=True)
sim = torch.nn.functional.cosine_similarity(q_emb, s_emb, dim=0)
print("\n Sanity check similarity (Q vs correct summary):", sim.item())

# STEP 7: Test retrieval (encode full summaries once)
summaries = df["full_summary"].dropna().astype(str).tolist()
summary_embeddings = model.encode(summaries, convert_to_tensor=True, normalize_embeddings=True)

# STEP 8: Example question → retrieve top 3 summaries
query = "قیمت تصفیه هوا چقدر است؟"
query_embedding = model.encode(query, convert_to_tensor=True, normalize_embeddings=True)

import torch
cosine_scores = torch.nn.functional.cosine_similarity(query_embedding, summary_embeddings)
top_results = torch.topk(cosine_scores, k=3)

for score, idx in zip(top_results.values, top_results.indices):
    print("\n Score:", score.item())
    print(" Summary:", summaries[idx])

Some weights of XLMRobertaModel were not initialized from the model checkpoint at cis-lmu/glot500-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Step,Training Loss



 Sanity check similarity (Q vs correct summary): 0.9809654951095581


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)



 Score: 0.982026219367981
 Summary: خلاصه نظرات:
نظر 1: از نظر عملکردش در تصفیه هوا نمیشه نظر داد چون نیازمند کار کارشناسی و آزمایشگاهیه. از نظر طراحی، ورودی هوا از بالاست و دو خروجی هوا در طرفینش داره. دو مشکل داره. یکی اینکه درب بالایی که برای تعویض فیلتر هست خیلی بیش از حد ساده و با فشار کم باز میشه و وقتی شما دستگاه رو در دست گرفتید و در حال حملش هستید ممکنه تصادفی فشار بیارید و درش باز بشه. ایراد دوم اینه که کار با دکمه ی دستگاه چندان راحت نیست و یکم قلق داره. یعنی شما فشار بدید ممکنه هیچ اتفاقی نیوفته. باید انگشتتون رو روش بکشید تا اثر کنه.
نظر 2: با درود بر شما عزیزان من این دستگاه رو ۱۰ بهمن سال ۹۹ خریدم و الان دقیقا حدود یک سال از خرید این دستگاه میگذره و بالا پایین این دستگاه رو تست کردم و نظر خودم رو قلبن، حقیقا خدمتتون عرض می کنم من این دستگاه رو درست در اواخر اوج آلودگی هوای تهران در اون سال خریدم و واقعا خودم انسانی هستم که توی هوای آلوده بالای ۱۵۰ تهران مشکل می خورم از بچگی و برام خرید این دستگاه برای اینکه بتونم کار بکنم خیلی مهم بود که هم در خودرو و هم در منزل بشود اس

In [ ]:
# STEP 8: Example question → retrieve top 3 summaries
query = "قیمت تصفیه هوا چقدر است؟"
query_embedding = model.encode(query, convert_to_tensor=True, normalize_embeddings=True)

import torch
cosine_scores = torch.nn.functional.cosine_similarity(query_embedding, summary_embeddings)
top_results = torch.topk(cosine_scores, k=3)

for score, idx in zip(top_results.values, top_results.indices):
    print("\n Score:", score.item())
    print(" Summary:", summaries[idx])

# Very Improved Fine-Tuned GLOT500 (could be our final code!)

In [ ]:
!pip install -q sentence-transformers joblib openpyxl

import os, re, json, math, numpy as np, pandas as pd, joblib, torch
from pathlib import Path
from typing import List, Dict, Tuple
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import cosine_similarity

from sentence_transformers import models, SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModel

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 91.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 828.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 54.2 MB/s eta 0:00:00


In [ ]:
os.environ["WANDB_DISABLED"] = "true"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

CSV_PATH = Path("highest_3000_info_data_QA_patched.csv")
OUT_DIR  = Path("./glot500_finetuned_mean")
CTX_FILE = Path("finetuned_glot_ctx_embeddings_len512.joblib")

EPOCHS   = 3
BATCH    = 16
FREEZE_N = 6

MAX_LEN  = 512
STRIDE   = 128

USE_PCA_WHITEN   = True
PCA_COMPONENTS   = 256
PCA_RANDOM_STATE = 0

In [ ]:
df = pd.read_csv(CSV_PATH)
assert "full_summary" in df.columns, "missing full_summary"
assert "product_id" in df.columns,   "missing product_id"
assert "qa_pairs" in df.columns,     "missing qa_pairs"

product_ids    = df["product_id"].astype(str).tolist()
full_summaries = df["full_summary"].astype(str).tolist()

In [ ]:
def iter_qapairs(raw_text: str):
    for block in str(raw_text).split("\n\n---\n\n"):
        lines = [ln for ln in block.strip().split("\n") if ln.strip()]
        q = next((ln.split(":",1)[1].strip() for ln in lines if ln.strip().startswith("Q")), None)
        a = next((ln.split(":",1)[1].strip() for ln in lines if ln.strip().startswith("A")), None)
        if q and a:
            yield q, a

train_pairs: List[InputExample] = []
for _, row in df.iterrows():
    summary = row.get("full_summary")
    qa_block = row.get("qa_pairs")
    if not isinstance(summary, str) or not isinstance(qa_block, str):
        continue
    for q, _a in iter_qapairs(qa_block):
        train_pairs.append(InputExample(texts=[q, summary]))
print(f"Training pairs: {len(train_pairs)}")

Training pairs: 14989


In [ ]:
word = models.Transformer("cis-lmu/glot500-base")
word.max_seq_length = MAX_LEN
pool = models.Pooling(
    word.get_word_embedding_dimension(),
    pooling_mode_mean_tokens=True,
    pooling_mode_cls_token=False,
    pooling_mode_max_tokens=False
)
st_model = SentenceTransformer(modules=[word, pool])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.58G [00:00<?, ?B/s]

Some weights of XLMRobertaModel were not initialized from the model checkpoint at cis-lmu/glot500-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/496 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/7.66M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

In [ ]:
hf = st_model._first_module().auto_model
for p in hf.embeddings.parameters():
    p.requires_grad = False
if hasattr(hf, "encoder") and hasattr(hf.encoder, "layer"):
    encoder_layers = hf.encoder.layer
    for i, layer in enumerate(encoder_layers):
        if i < FREEZE_N:
            for p in layer.parameters():
                p.requires_grad = False
    print(f"Frozen: embeddings + first {FREEZE_N} layers")
else:
    print("Could not access encoder layers; skipping freeze")

Frozen: embeddings + first 6 layers


In [ ]:
train_loader = DataLoader(train_pairs, shuffle=True, batch_size=BATCH)
loss_fn = losses.MultipleNegativesRankingLoss(st_model)
warmup = max(100, int(0.1 * (len(train_loader) * EPOCHS)))

st_model.fit(
    train_objectives=[(train_loader, loss_fn)],
    epochs=EPOCHS,
    warmup_steps=warmup,
    output_path=str(OUT_DIR),
    show_progress_bar=True,
)
print("Fine-tuned ST saved to:", OUT_DIR)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Step,Training Loss
500,1.041400
1000,0.294200
1500,0.215700
2000,0.171600
2500,0.145700


Fine-tuned ST saved to: glot500_finetuned_mean


In [ ]:
OUT_DIR = "./glot500_finetuned_mean"
tokenizer = AutoTokenizer.from_pretrained(OUT_DIR, local_files_only=True)
hf_model  = AutoModel.from_pretrained(OUT_DIR, local_files_only=True).to(DEVICE).eval()

def embed_texts(texts: List[str], max_len=128, batch=32) -> np.ndarray:
    vecs = []
    for i in range(0, len(texts), batch):
        enc = tokenizer(
            texts[i:i+batch],
            padding=True, truncation=True, max_length=max_len,
            return_tensors="pt"
        ).to(DEVICE)
        with torch.no_grad():
            hs   = hf_model(**enc).last_hidden_state
            mask = enc["attention_mask"].unsqueeze(-1)
            vec  = (hs * mask).sum(dim=1) / mask.sum(dim=1)
        vecs.append(vec.detach().cpu())
    return torch.cat(vecs, dim=0).numpy()

def embed_doc_sliding(text: str, max_len=512, stride=128, batch=16) -> np.ndarray:
    enc = tokenizer(
        text,
        max_length=max_len,
        truncation=True,
        stride=stride,
        padding="max_length",
        return_overflowing_tokens=True,
        return_tensors="pt"
    )
    tok_counts = enc["attention_mask"].sum(dim=1).cpu().numpy().astype(np.float32)
    n_chunks   = enc["input_ids"].size(0)
    chunk_vecs = []
    for i in range(0, n_chunks, batch):
        inp = {
            "input_ids":      enc.input_ids[i:i+batch].to(DEVICE),
            "attention_mask": enc.attention_mask[i:i+batch].to(DEVICE)
        }
        with torch.no_grad():
            hs   = hf_model(**inp).last_hidden_state
            mask = inp["attention_mask"].unsqueeze(-1)
            vec  = (hs * mask).sum(1) / mask.sum(1)
        chunk_vecs.append(vec.detach().cpu().numpy())
    V = np.vstack(chunk_vecs)
    w = tok_counts / (tok_counts.sum() if tok_counts.sum() else 1.0)
    return (V * w[:, None]).sum(0)

def embed_corpus_sliding(texts: List[str], max_len=512, stride=128) -> np.ndarray:
    return np.vstack([embed_doc_sliding(t, max_len=max_len, stride=stride) for t in texts])

if CTX_FILE.exists():
    ctx_embs = joblib.load(CTX_FILE)
    print("Loaded ctx embeddings:", CTX_FILE)
else:
    print("Encoding summaries (sliding-window, finetuned weights)…")
    ctx_embs = embed_corpus_sliding(full_summaries, max_len=MAX_LEN, stride=STRIDE)
    ctx_embs = normalize(ctx_embs)  # L2 normalize rows
    joblib.dump(ctx_embs, CTX_FILE)
    print("Saved ctx embeddings →", CTX_FILE)

Loaded ctx embeddings: finetuned_glot_ctx_embeddings_len512.joblib


In [ ]:
if USE_PCA_WHITEN:
    pca = PCA(n_components=PCA_COMPONENTS, whiten=True, random_state=PCA_RANDOM_STATE)
    ctx_proj = pca.fit_transform(ctx_embs)
    ctx_proj = normalize(ctx_proj)

    def embed_question_proj(q: str) -> np.ndarray:
        qv = embed_texts([q], max_len=MAX_LEN)[0]
        qv = normalize(pca.transform([qv]))
        return qv
else:
    pca = None
    ctx_proj = ctx_embs

    def embed_question_proj(q: str) -> np.ndarray:
        qv = embed_texts([q], max_len=MAX_LEN)[0]
        qv = normalize([qv])
        return qv

In [ ]:
def parse_qa_pairs(raw_text: str):
    pattern = re.compile(
        r"""^\s*
            \d+-\s*
            (?P<q>.+?)
            \s*\n
            ID:\s*
            (?P<pid>\d+)\s*
            (?:\nپاسخ:.*?)?
            (?=\n\d+-|\Z)
        """,
        re.MULTILINE | re.VERBOSE | re.DOTALL
    )
    return [
        {"question": m.group("q").strip(), "product_id": m.group("pid")}
        for m in pattern.finditer(raw_text)
    ]

qa_str = '''1- کدام ضدآفتاب پوست رو به شدت سفید میکنه؟
ID: 7324503
پاسخ: کرم ضد آفتاب بی رنگ مای SPF50 مدل Hyaluronic Acid مناسب پوست های چرب حجم 50 میلی لیتر به همراه فلوئید ضد آفتاب مای مدل Hypoallergen حجم 50 میلی لیتر
2- کدام داستان به روابط خواهر برادری می پردازد؟
ID: 2849263
پاسخ: کتاب تابستانی به رنگ اقیانوس اثر جیلیان مک دان انتشارات پرتقال
3- کدام تونر پاک کننده صورت برای پوست های چرب و جوش دار عالیه؟
ID: 4948922
پاسخ: تونر پاک کننده صورت وان سپرینگ مدل POMEGRANATE حجم 150 میلی لیتر
4- کدام ژل رنگ مو برای موی کوتاه مناسب نیست؟
ID: 4541218
پاسخ: ژل رنگ مو جنتالین مدل 08 حجم 100 میلی لیتر رنگ سفید
5- کدام گیتار کلاسیک برای مبتدی ها از همه جهت عالیه؟
ID: 2707861
پاسخ: گیتار کلاسیک کینگ مدل K2
6- کدام ماشین اصلاح موی صورت با مدلهای قبلیش زیاد فرقی نداره؟
ID: 1608269
پاسخ: ماشین اصلاح موی صورت پاناسونیک مدل ES-LV9QX
7- کدام ساعت مچی نیاز به عوض کردن باتری نداره؟
ID: 296557
پاسخ: ساعت مچی عقربه ای مردانه کاسیو مدل AQ-S810W-1A3VDF
8- کدام بازی تکرار پذیری زیادی داره؟
ID: 1551517
پاسخ: بازی فکری نکست موو مدل Azul
9- کدام عطر مردانه از نظرات مشخصه که فیکه؟
ID: 54972
پاسخ: ادو تویلت مردانه کارولینا هررا مدل 212 حجم 100 میلی لیتر
10- کدام لیوان بسیار نازک است و جنس ضعیفی دارد؟
ID: 3179643
پاسخ: لیوان سفری مدل STL 9X
11- کدام کاپشن زنانه مناسب هوای سرد نیست؟
ID: 10729336
پاسخ: کاپشن زنانه السانا مدل فرینا کد 137101
12- کدام ست برس برای افراد غیر حرفه ای مناسب است؟
ID: 3415330
پاسخ: ست برس آرایشی مدل B12 مجموعه 12 عددی
13- کدام ملودیکا دوام زیادی برای استفاده طولانی مدت نداره؟
ID: 77308
پاسخ: ملودیکا هوهنر مدل Superforce 37
14- کدام محصول براون ساده ترین دستگاه این برند است؟
ID: 479189
پاسخ: اپیلاتور براون سری Silk Epil1 مدل 1170
15- کدام محصول لایه بردار سطحی با اثر ضدچروک و روشن کنندگی است؟
ID: 4474244
پاسخ: ژل لایه بردار دلانو مدل 5464 حجم 50 میلی لیتر
16- کدام زعفران هیچ آرم و مارکی ندارد؟
ID: 8476294
پاسخ: زعفران درجه یک زرتاج - 4.608 گرم
17- کدام اسپری دوفاز بوی نامناسبی دارد؟
ID: 2411616
پاسخ: اسپری دوفاز بیول مدل Argan حجم 450 میلی لیتر
18- کدام ساعت مچی دیجیتال ساعت خوبیه نسبت به قیمتش؟
ID: 4265694
پاسخ: ساعت مچی دیجیتال مردانه اسکمی مدل 1358M
19- کدام دستگاه تصفیه کننده هوا سایزش بسیار کوچیکه؟
ID: 4195578
پاسخ: دستگاه تصفیه کننده هوا مدل XP02
20- کدام شامپو زیاد کف نمیکند و بدون سولفات  است؟
ID: 9117862
پاسخ: شامپو و نرم کننده مو ثمین مدل اوره حجم 200 میلی لیتر
21- کدام ادو پرفیوم مردانه مشابه برند اصلی است؟
ID: 11091234
پاسخ: ادو پرفیوم مردانه روونا مدل مگامار حجم 100 میلی لیتر
22- کدام سرم پوست لایه برداری ملایم است؟
ID: 5054238
پاسخ: سرم پوست دی اوردینری مدل Lactic Acid حجم 30 میلی لیتر
23- کدام کرم مرطوب کننده خوراک پوست خشک است؟
ID: 321110
پاسخ: کرم مرطوب کننده اوریاژ سری EAU Thermale مدل Rich Water حجم 40 میلی لیتر
24- کدام بازی فکری مفرح و جذاب برای سنین مختلف است؟
ID: 4173999
پاسخ: بازی فکری مدل فرار از جهنم
25- کدام کنسول بازی کلی بازی نوستالژی داره؟
ID: 6880355
پاسخ: مجموعه کنسول بازی مدل Anniversary Edition 620 Games Built In-Classic
26- کدام کمربند مردانه چرم طبیعی نیست؟
ID: 3621923
پاسخ: کمربند مردانه مدل چرم بارز DM20M
27- کدام بازی فکری بازطراحی از بازی مسترجک پاکت هست؟
ID: 2676959
پاسخ: بازی فکری جک قاتل مدل 001
28- کدام ماسک مو برای جلوگیری از وزکردن مو خوبه؟
ID: 1496724
پاسخ: ماسک مو آدرا مدل Anti Frizz حجم 80 میلی لیتر
29- کدام سیم ویولن اوریجینال واقعی می باشد؟
ID: 3555956
پاسخ: سیم ویولن پیراسترو سبز مدل chromcor
30- کدام عینک آفتابی در هیچ جای شیشه و فریم مارک برند رو نداره؟
ID: 397670
پاسخ: عینک آفتابی رین بی مدل 8747GR
31- کدام ادو پرفیوم بوی بتادین دارد؟
ID: 7501023
پاسخ: ادو پرفیوم فراگرنس ورد مدل Barakkat ROUGE 540 حجم 100 میلی لیتر
32- کدام کیف رودوشی چرم خوش دوخت و برای هدیه دادن مناسبه؟
ID: 681604
پاسخ: کیف رودوشی چرم طبیعی چرم بارز مدل DG39
33- کدام ساک سازگار با محیط زیست است؟
ID: 3507286
پاسخ: ساک خرید خندالو طرح پرنده ای در قفس کد 256
34- کدام کتاب وقایع رو خیلی خوب توصیف کرده؟
ID: 202727
پاسخ: کتاب زندگی دوم اثر اس.جی.واتسون
35- کدام بخور سرد قابل اطمینان نیست؟
ID: 295042
پاسخ: بخور سرد و گرم زیکلاس مد مدل Jss-37501
36- کدام نیم بوت کیفیت معمولی دارد و برای کوهنوردی خوب است؟
ID: 991856
پاسخ: نیم بوت مردانه و زنانه نسیم مدل تورین کد 3405
37- کدام محصول برای طبیعت گردی و کوهنوردی در ۶ ماه اول سال عالی است؟
ID: 2225006
پاسخ: کیسه خواب ردکمپ مدل G200
38- کدام تشک برای بیماری که نمیتواند حرکت کند مناسب است؟
ID: 3676786
پاسخ: تشک مواج ضد زخم بستر زنیتمد مدل NT801
39- کدام شامپو تاریخ و برچسب اصالت ندارد؟
ID: 3228539
پاسخ: شامپو او جی ایکس مدل Brazilian Keratin Smooth حجم 385 میلی لیتر
40- کدام محصول کم صداست و زود هم داغ نمیشه؟
ID: 3671833
پاسخ: بند انداز برقی اکسیژن مدل Ox-9973
41- کدام کتاب مربوط به فاشیسم و صهیونیسم است؟
ID: 4565008
پاسخ: کتاب اسلام گرایی اثر ارنست نولته نشر ثالث
42- کدام کیف اداری نسبت به کیف های چرم مصنوعی بازار خیلی بهتره؟
ID: 332406
پاسخ: کیف اداری کهن چرم مدل L73-50
43- کدام اسباب بازی شبیه عکسش نیست؟
ID: 10289863
پاسخ: اسباب بازی مدل پمپ بنزین دو نازله
44- کدام کیت آموزشی برای کادو به یک کودک 10 سال مناسب است؟
ID: 864225
پاسخ: کیت آموزشی ربات خورشیدی سولار پاور مدل سفینه فضایی
45- کدام رژ لب مایع جلوه ی خاصی به آرایش میده؟
ID: 254047
پاسخ: رژ لب مایع لنسور سری Glaze شماره 02
46- کدام اتوی موی چرخشی بدون حرارت زیاد مو رو صاف میکنه؟
ID: 738429
پاسخ: اتو موی چرخشی استایلر مدل IS1001
47- کدام تونر صورت پوست رو تمیز و صاف میکنه؟
ID: 10555180
پاسخ: تونر صورت بیتروی مدل 01 حجم 200 میلی لیتر
48- کدام نیم تنه ورزشی خوش فرم و سبک و راحته:
ID: 522392
پاسخ: نیم تنه ورزشی زنانه آریو شاپ مدل 131
49- کدام پیراهن زنانه برای قد بلند کاملا مناسبه؟
ID: 8696531
پاسخ: پیراهن زنانه مدل D03
50- کدام محصول برای چمدان بزرگ عالیه؟
ID: 9195218
پاسخ: کاور چمدان مدل رویال سایز بزرگ'''
eval_pairs = parse_qa_pairs(qa_str)
print(f"Eval questions parsed: {len(eval_pairs)}")

pid_to_idx: Dict[str, int] = {}
for idx, pid in enumerate(product_ids):
    pid_to_idx.setdefault(pid, idx)

Eval questions parsed: 50


In [ ]:
def retrieve_topk_finetuned(question: str, top_k: int = 3):
    qv   = embed_question_proj(question)    # shape (1, d)
    sims = (qv @ ctx_proj.T)[0]             # (N,)
    idxs = np.argsort(sims)[-top_k:][::-1]
    return [
        {
            "rank": i+1,
            "score": float(sims[idx]),
            "product_id": product_ids[idx],
            "context": full_summaries[idx]
        }
        for i, idx in enumerate(idxs)
    ]

rows_top3 = []
for pair in eval_pairs:
    q        = pair["question"]
    gold_pid = str(pair["product_id"])
    top3 = retrieve_topk_finetuned(q, top_k=3)

    gold_rank = 0
    for item in top3:
        if item["product_id"] == gold_pid:
            gold_rank = item["rank"]
            break

    for item in top3:
        rows_top3.append({
            "question": q,
            "rank": item["rank"],
            "score": round(item["score"], 4),
            "product_id": item["product_id"],
            "context": item["context"],
            "gold_pid": gold_pid,
            "gold_in_top3": 1 if gold_rank else 0,
            "gold_rank": gold_rank if gold_rank else ""
        })

out_xlsx = "finetuned_glot_top3_contexts.xlsx"
pd.DataFrame(rows_top3).to_excel(out_xlsx, index=False, engine="openpyxl")
print("Finally Saved:", out_xlsx)

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/usr/local/l

Finally Saved: finetuned_glot_top3_contexts.xlsx
